In [139]:
#!/usr/bin/env python


# coding: utf-8
from __future__ import unicode_literals
import sys
try:
    import requests
except ImportError:
    !python2 -m pip install requests
    import requests
    
import json
import time
import codecs
import sys
try:
    import pandas as pd 
except ImportError:
    !python2 -m pip install pandas
    import pandas as pd
try:
    import numpy as np
except ImportError:
    !python2 -m pip install numpy
    import numpy as np

from itertools import izip #iter pairwise
from itertools import izip_longest #iter over tuples

In [140]:
#import yaml #alternative solution for json probs
#UTF8Writer = codecs.getwriter('utf8') #toUtf8 stuffs, nevermind
#sys.stdout = UTF8Writer(sys.stdout)

In [141]:
debug=1
#debug=0
debug_count=1
count_stopper=100

In [142]:
member_df=pd.read_csv("../csv/struttura/member.csv")
print member_df.head(2)

   member_id    member_name
0  220200963    Sarah Roxas
1  253755396  Lupco Trpeski


In [143]:
member_ids=member_df['member_id']
member_df['topics']="err"
member_df['lat']=0.
member_df['long']=0.
print member_df.head()

   member_id           member_name topics  lat  long
0  220200963           Sarah Roxas    err  0.0   0.0
1  253755396         Lupco Trpeski    err  0.0   0.0
2  272979286             Malikmika    err  0.0   0.0
3          6       Scott Heiferman    err  0.0   0.0
4  232783879  Luis Eduardo Saltini    err  0.0   0.0


In [144]:
#remember to hide it after 
fabri_key="2e12625a12642d6ac743d19566c393e"
max_key="445811b5a6b424f7e79342826176d"
my_api_key= "546938372953301546964e404246e"

In [145]:
#not really working but who cares 
block_alert={u'problem': u'Client throttled', u'code': u'throttled', u'details': u'Credentials have been throttled'}

In [146]:
#iterating tuples at once
#old pairs version (to keep in case)
def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return izip(a, a)

In [147]:
#more complete groupin version
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
    args = [iter(iterable)] * n
    return izip_longest(fillvalue=fillvalue, *args)

In [148]:
def main():
        count=0
        tot_count=0
        print "requesting at http://api.meetup.com/2/members"        

        #for line in member_df.itertuples():           #basic iter
        #for x, y in pairwise(member_df.itertuples()): #pairs iter
        for z in grouper(member_df.itertuples(),3): #groups
            
            #request parameters	
            per_page = 1
            #results_we_got = per_page #more pages output
            offset = 0            
       
            #get id (one should really vectorize it)
            id_0=member_df.iloc[z[0].Index]['member_id']
            id_1=member_df.iloc[z[1].Index]['member_id']
            id_2=member_df.iloc[z[2].Index]['member_id']

            # Meetup.com documentation here: http://www.meetup.com/meetup_api/docs/2/groups/
            try:    
                response0=get_results({"member_id":id_0, "key":max_key, "page":per_page, "offset":offset}, tot_count,1)
                time.sleep(0.05) #PLS U NO BAN me
            
                response1=get_results({"member_id":id_1, "key":my_api_key, "page":per_page, "offset":offset}, tot_count,2)
                time.sleep(0.05) #PLS U NO BAN me
            
                response2=get_results({"member_id":id_2, "key":fabri_key, "page":per_page, "offset":offset}, tot_count,3)
            except Exception as e:
                print "exception encountered at requesting: "
                print e
            time.sleep(0.15) #PLS U NO BAN me
            offset += 1
            #results_we_got = response['meta']['count']              
            #time.sleep(1)
            count+=1
            tot_count+=1
            if debug:
                print "resp 0"
                print response0
                print "resp 1"
                print response1
                print "resp 2"
                print response2
            
            #one should really loop over those
            try:
                data0=response0['results']

                for elem in data0:
                    if elem['topics']:
                        #print elem['topics']
                        topic_list=[]
                        for e in elem['topics']:
                            a=json.dumps(e)
                            b=json.loads(a) #unicode prob
                            #print b['urlkey']
                            topic_list.append(str(b['urlkey']).encode('utf-8'))
                            #print topic_list

                            #c= yaml.safe_load(a) #alternative in case of utf8 representation, keep it just as a reminder
                            #print c['urlkey']

                        #member_df['topics'].at[z[0].Index]=elem['topics'] #get the whole list of dicts 
                        member_df['topics'].at[z[0].Index]=topic_list #get only the topic urlkey list
                    else:
                        member_df['topics'].at[z[0].Index]="NaN"
                    try:
                        if elem['lon']:
                                if elem['lat']:
                                    member_df['long'].at[z[0].Index]=elem['lon']
                                    member_df['lat'].at[z[0].Index]=elem['lat']
                        else:
                            member_df['long'].at[z[0].Index]="NaN"
                            member_df['lat'].at[z[0].Index]="NaN"
                    except: 
                        pass #for the moment lets try this way
                        
            except Exception as e:
                print "exception in response0: "
                print e
                print "is the "+str(count)+"-th iteration"
                try: 
                    print "response is: "
                    print response0
                except:
                    pass
                                                    
            try:
                data1=response1['results']

                for elem in data1:
                    if elem['topics']:
                        #print elem['topics']
                        topic_list=[]
                        for e in elem['topics']:
                            a=json.dumps(e)
                            b=json.loads(a) #unicode prob
                            #print b['urlkey']
                            topic_list.append(str(b['urlkey']).encode('utf-8'))
                            #print topic_list

                            #c= yaml.safe_load(a) #alternative in case of utf8 representation, keep it just as a reminder
                            #print c['urlkey']

                        #member_df['topics'].at[z[0].Index]=elem['topics']
                        member_df['topics'].at[z[1].Index]=topic_list
                    else:
                        member_df['topics'].at[z[1].Index]="NaN"
                    try:
                        if elem['lon']:
                                if elem['lat']:
                                    member_df['long'].at[z[1].Index]=elem['lon']
                                    member_df['lat'].at[z[1].Index]=elem['lat']
                        else:
                            member_df['long'].at[z[1].Index]="NaN"
                            member_df['lat'].at[z[1].Index]="NaN"
                    except: 
                        pass #for the moment lets try this way

            except Exception as e:
                print "exception in response1: "
                print e
                print "is the "+str(count)+"-th iteration"
            try:                                        
                data2=response2['results']

                for elem in data2:
                    if elem['topics']:
                        #print elem['topics']
                        topic_list=[]
                        for e in elem['topics']:
                            a=json.dumps(e)
                            b=json.loads(a) #unicode prob
                            #print b['urlkey']
                            topic_list.append(str(b['urlkey']).encode('utf-8'))
                            #print topic_list

                            #c= yaml.safe_load(a) #alternative in case of utf8 representation, keep it just as a reminder
                            #print c['urlkey']

                        #member_df['topics'].at[z[0].Index]=elem['topics']
                        member_df['topics'].at[z[2].Index]=topic_list
                    else:
                        member_df['topics'].at[z[2].Index]="NaN"
                    try:
                        if elem['lon']:
                                if elem['lat']:
                                    member_df['long'].at[z[2].Index]=elem['lon']
                                    member_df['lat'].at[z[2].Index]=elem['lat']
                        else:
                            member_df['long'].at[z[2].Index]="NaN"
                            member_df['lat'].at[z[2].Index]="NaN"
                    except: 
                        pass #for the moment lets try this way

            except Exception as e:
                print "exception in response2: "
                print e
                print "is the "+str(count)+"-th iteration"
            
            if response0 is block_alert:
                print "throttle alert on response 0 (max key)"
                break
            if response1 is block_alert:
                print "throttle alert on response 1 (my key)"
                break
            if response2 is block_alert:
                print "throttle alert on response 2 (fabri key)"
                break
            if debug_count:
                if tot_count is count_stopper:
                    print "breaking"
                    break
                if count is 50000:
                    print "partial count is"+ str(count)
                    print "csv printed at iteration" + str(tot_count)+" to be multiplied *3 btw"
                    member_df.to_csv("../csv/struttura/member_enriched_easter.csv")
                    count=0
            
        print member_df.head()
        print member_df.tail()
        print "exporting to csv"
        member_df.to_csv("../csv/struttura/member_enriched_final.csv") #prolly never reach here sadly but we always hope that member_enriched_final.csv will be identical to member_enriched.csv, that would be mad&amazing

In [149]:
def get_results(params, tot_count, n):

    request = requests.get("http://api.meetup.com/2/members",params=params)
    try:
        data = request.json()
        #iter_data=json.load(data)
        #a=data['results']
        #for elem in a:
            #print(elem['topics'])
        #print(request)
    except Exception as e:
        print "trying again:"
        time.sleep(0.05)
        try:
            request = requests.get("http://api.meetup.com/2/members",params=params)
            data = request.json()
            print "noice"
        except Exception as e:
            print "failed anyway"
            print "internal exception in loading request.json() at the "+ str(tot_count*3+n) + "iteration"
            print e
            print request
            print request.content
            pass
        pass
        
        
     
    return data


# In[123]:

In [150]:
if __name__=="__main__":
        main()

requesting at http://api.meetup.com/2/members
trying again:
noice
resp 0
{u'meta': {u'count': 1, u'updated': 1513183560000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?member_id=220200963&offset=0&format=json&page=1&key=445811b5a6b424f7e79342826176d&order=name', u'total_count': 1, u'lon': u'', u'next': u'', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'Gen\xe8ve', u'name': u'Sarah Roxas', u'other_services': {}, u'country': u'ch', u'topics': [], u'lon': 6.14, u'joined': 1484745115000, u'id': 220200963, u'link': u'http://www.meetup.com/members/220200963', u'photo': {u'thumb_link': u'https://secure.meetupstatic.com/photos/member/e/9/8/b/thumb_263459787.jpeg', u'photo_id': 263459787, u'highres_link': u'https://secure.meetupstatic.com/photos/member/e/9/8/b/highres_263459787.jpeg', u'base_url': u'h

resp 0
{u'meta': {u'count': 1, u'updated': 1548642325000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?member_id=260046859&offset=0&format=json&page=1&key=445811b5a6b424f7e79342826176d&order=name', u'total_count': 1, u'lon': u'', u'next': u'', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'Perth', u'name': u'Sikares Pholhem', u'other_services': {}, u'country': u'au', u'topics': [{u'name': u'Startup Businesses', u'urlkey': u'startup-businesses', u'id': 21681}, {u'name': u'Business Strategy', u'urlkey': u'business-strategy', u'id': 17635}, {u'name': u'Web Design', u'urlkey': u'webdesign', u'id': 659}, {u'name': u'Lean Startup', u'urlkey': u'lean-startup', u'id': 38660}, {u'name': u'Real Estate Investors', u'urlkey': u'real-estate-investors', u'id': 16165}, {u'name': u'Small Business Owners', u'ur

resp 0
{u'meta': {u'count': 1, u'updated': 1554756794000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?member_id=278630976&offset=0&format=json&page=1&key=445811b5a6b424f7e79342826176d&order=name', u'total_count': 1, u'lon': u'', u'next': u'', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'S\xe3o Paulo', u'name': u'Vanessa Lima dos Santos', u'other_services': {}, u'country': u'br', u'topics': [], u'lon': -46.63, u'joined': 1554756784000, u'id': 278630976, u'link': u'http://www.meetup.com/members/278630976', u'photo': {u'thumb_link': u'https://secure.meetupstatic.com/photos/member/d/5/1/f/thumb_286974559.jpeg', u'photo_id': 286974559, u'highres_link': u'https://secure.meetupstatic.com/photos/member/d/5/1/f/highres_286974559.jpeg', u'base_url': u'https://secure.meetupstatic.com', u'type': u'membe

trying again:
noice
resp 0
{u'meta': {u'count': 1, u'updated': 1509331087000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?member_id=5167593&offset=0&format=json&page=1&key=445811b5a6b424f7e79342826176d&order=name', u'total_count': 1, u'lon': u'', u'next': u'', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'Long Beach', u'photo': {u'thumb_link': u'https://secure.meetupstatic.com/photos/member/f/8/0/thumb_258783968.jpeg', u'photo_id': 258783968, u'highres_link': u'https://secure.meetupstatic.com/photos/member/f/8/0/highres_258783968.jpeg', u'base_url': u'https://secure.meetupstatic.com', u'type': u'member', u'photo_link': u'https://secure.meetupstatic.com/photos/member/f/8/0/member_258783968.jpeg'}, u'name': u'Lillian', u'bio': u'Meetup is fun with friends and all the acitivites/hobbies it has t

resp 0
{u'meta': {u'count': 1, u'updated': 1528726512000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?member_id=254454453&offset=0&format=json&page=1&key=445811b5a6b424f7e79342826176d&order=name', u'total_count': 1, u'lon': u'', u'next': u'', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'Petakh Tiqwa', u'name': u'Hila Zarosim', u'other_services': {}, u'country': u'il', u'topics': [], u'lon': 34.88, u'joined': 1526287100000, u'id': 254454453, u'link': u'http://www.meetup.com/members/254454453', u'photo': {u'thumb_link': u'https://secure.meetupstatic.com/photos/member/d/0/8/d/thumb_277973389.jpeg', u'photo_id': 277973389, u'highres_link': u'https://secure.meetupstatic.com/photos/member/d/0/8/d/highres_277973389.jpeg', u'base_url': u'https://secure.meetupstatic.com', u'type': u'member', u'photo_

...

trying again:
noice
resp 0
{u'meta': {u'count': 1, u'updated': 1538093138000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?member_id=264241791&offset=0&format=json&page=1&key=445811b5a6b424f7e79342826176d&order=name', u'total_count': 1, u'lon': u'', u'next': u'', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'Spencerport', u'name': u'Joyce Westbrook', u'other_services': {}, u'country': u'us', u'topics': [{u'name': u'Hip Hop', u'urlkey': u'hiphop', u'id': 128}, {u'name': u'Wine', u'urlkey': u'wine', u'id': 611}, {u'name': u'Beer', u'urlkey': u'beer', u'id': 1475}, {u'name': u'Self-Empowerment', u'urlkey': u'self-empowerment', u'id': 38757}, {u'name': u'Culture Exchange', u'urlkey': u'culture-exchange', u'id': 67622}, {u'name': u'Self-Improvement', u'urlkey': u'self-improvement', u'id': 3336}, {u

resp 0
{u'meta': {u'count': 1, u'updated': 1554087285000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?member_id=264940311&offset=0&format=json&page=1&key=445811b5a6b424f7e79342826176d&order=name', u'total_count': 1, u'lon': u'', u'next': u'', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'Minneapolis', u'name': u'Sam L.', u'other_services': {}, u'country': u'us', u'topics': [], u'lon': -93.27, u'joined': 1538849925000, u'id': 264940311, u'state': u'MN', u'link': u'http://www.meetup.com/members/264940311', u'photo': {u'thumb_link': u'https://secure.meetupstatic.com/photos/member/5/2/c/b/thumb_286521195.jpeg', u'photo_id': 286521195, u'highres_link': u'https://secure.meetupstatic.com/photos/member/5/2/c/b/highres_286521195.jpeg', u'base_url': u'https://secure.meetupstatic.com', u'type': u'member

resp 0
{u'meta': {u'count': 1, u'updated': 1554145883000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?member_id=78643312&offset=0&format=json&page=1&key=445811b5a6b424f7e79342826176d&order=name', u'total_count': 1, u'lon': u'', u'next': u'', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'Vancouver', u'name': u'Divya', u'other_services': {}, u'country': u'ca', u'topics': [], u'lon': -123.12, u'joined': 1359608187000, u'id': 78643312, u'state': u'BC', u'link': u'http://www.meetup.com/members/78643312', u'photo': {u'thumb_link': u'https://secure.meetupstatic.com/photos/member/6/e/e/7/thumb_275548391.jpeg', u'photo_id': 275548391, u'highres_link': u'https://secure.meetupstatic.com/photos/member/6/e/e/7/highres_275548391.jpeg', u'base_url': u'https://secure.meetupstatic.com', u'type': u'member', u'

resp 0
{u'meta': {u'count': 1, u'updated': 1555650946000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?member_id=90178242&offset=0&format=json&page=1&key=445811b5a6b424f7e79342826176d&order=name', u'total_count': 1, u'lon': u'', u'next': u'', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'Montr\xe9al', u'name': u'Louis', u'other_services': {}, u'country': u'ca', u'topics': [{u'name': u'Language & Culture', u'urlkey': u'language', u'id': 10454}, {u'name': u'Outdoors', u'urlkey': u'outdoors', u'id': 15046}, {u'name': u'Storytelling', u'urlkey': u'storytelling', u'id': 9429}, {u'name': u'Self-Improvement', u'urlkey': u'self-improvement', u'id': 3336}, {u'name': u'Healthy Living', u'urlkey': u'healthy-living', u'id': 20221}, {u'name': u'Quantified Self', u'urlkey': u'quantified-self', u'id': 24099}

resp 0
{u'meta': {u'count': 1, u'updated': 1551270091000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?member_id=272630478&offset=0&format=json&page=1&key=445811b5a6b424f7e79342826176d&order=name', u'total_count': 1, u'lon': u'', u'next': u'', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'Johannesburg', u'name': u'Yu,Ke', u'other_services': {}, u'country': u'za', u'topics': [{u'name': u'Indian Culture', u'urlkey': u'indian-culture', u'id': 27904}, {u'name': u'Reading', u'urlkey': u'reading', u'id': 15360}, {u'name': u'Self-Improvement', u'urlkey': u'self-improvement', u'id': 3336}, {u'name': u'Fiction', u'urlkey': u'fiction', u'id': 15113}, {u'name': u'Courses and Workshops', u'urlkey': u'courses-and-workshops', u'id': 20629}, {u'name': u'Home Improvement', u'urlkey': u'home-improvement', u'id